# [모의 캐글-의료] 흉부 CT 코로나 감염 여부 분류
- 이미지 binary 분류 과제
- 담당: 이녕민M

## Import Libraries

In [1]:
#!apt-get update && apt-get install -y python3-opencv

In [2]:
#!pip install sklearn

In [1]:
import os, torch, copy, cv2, sys, random
# from datetime import datetime, timezone, timedelta
from PIL import Image
import numpy as np
import pandas as pd
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms

#MBConvBlock
import math
from functools import partial
from torch.nn import functional as F

#ScaledDotScaling
from torch.nn import init

In [2]:
import wandb

In [4]:
wandb.init(project="my-test-project", entity="jch")

In [5]:
wandb.config = {
  "learning_rate": 0.0005,
  "epochs": 50,
  "batch_size": 32
}

## Set Arguments & hyperparameters

In [6]:
# 시드(seed) 설정

RANDOM_SEED = 2022

torch.manual_seed(RANDOM_SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)

In [7]:
# parameters

### 데이터 디렉토리 설정 ###
DATA_DIR= 'data'
NUM_CLS = 2

EPOCHS = 50
BATCH_SIZE = 32 #32
LEARNING_RATE = 0.0005 #0.0005
EARLY_STOPPING_PATIENCE = 10
INPUT_SHAPE = 128 # 128

os.environ["CUDA_VISIBLE_DEVICES"]="0"
# DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
DEVICE=torch.device('cpu')

## Dataloader

#### Train & Validation Set loader

In [8]:
class CustomDataset(Dataset):
    def __init__(self, data_dir, mode, input_shape):
        self.data_dir = data_dir
        self.mode = mode
        self.input_shape = input_shape
        
        # Loading dataset
        self.db = self.data_loader()
        
        # Dataset split
        if self.mode == 'train':
            self.db = self.db[:int(len(self.db) * 0.9)]
        elif self.mode == 'val':
            self.db = self.db[int(len(self.db) * 0.9):]
            self.db.reset_index(inplace=True)
        else:
            print(f'!!! Invalid split {self.mode}... !!!')
            
        # Transform function
        self.transform = transforms.Compose([transforms.Resize(self.input_shape),
                                             transforms.ToTensor(),
                                             transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

    def data_loader(self):
        print('Loading ' + self.mode + ' dataset..')
        if not os.path.isdir(self.data_dir):
            print(f'!!! Cannot find {self.data_dir}... !!!')
            sys.exit()
        
        # (COVID : 1, No : 0)
        db = pd.read_csv(os.path.join(self.data_dir, 'train.csv'))
        
        return db

    def __len__(self):
        return len(self.db)

    def __getitem__(self, index):
        data = copy.deepcopy(self.db.loc[index])

        # Loading image
        cvimg = cv2.imread(os.path.join(self.data_dir,'train',data['file_name']), cv2.IMREAD_COLOR | cv2.IMREAD_IGNORE_ORIENTATION)
        if not isinstance(cvimg, np.ndarray):
            raise IOError("Fail to read %s" % data['file_name'])

        # Preprocessing images
        trans_image = self.transform(Image.fromarray(cvimg))
        return trans_image, data['COVID']


In [9]:
!pip install git+https://github.com/rwightman/pytorch-image-models.git

  Cloning https://github.com/rwightman/pytorch-image-models.git to /tmp/pip-req-build-bg4qy3_t
  Created wheel for timm: filename=timm-0.5.5-py3-none-any.whl size=432802 sha256=e7f62f3a8537bf893e634e012cdc3515def15a130fad23edbb0432a294eaa4c1
  Stored in directory: /tmp/pip-ephem-wheel-cache-lf7vjwez/wheels/b3/03/6a/79956ddc149294ccf13727ca946e8baf38ccbe593299074e86
Successfully built timm


In [10]:
# import math
# from functools import partial

# import torch
# from torch import nn
# from torch.nn import functional as F

# class SwishImplementation(torch.autograd.Function):
#     @staticmethod
#     def forward(ctx, i):
#         result = i * torch.sigmoid(i)
#         ctx.save_for_backward(i)
#         return result

#     @staticmethod
#     def backward(ctx, grad_output):
#         i = ctx.saved_variables[0]
#         sigmoid_i = torch.sigmoid(i)
#         return grad_output * (sigmoid_i * (1 + i * (1 - sigmoid_i)))

# class MemoryEfficientSwish(nn.Module):
#     def forward(self, x):
#         return SwishImplementation.apply(x)


# def drop_connect(inputs, p, training):
#     """ Drop connect. """
#     if not training: return inputs
#     batch_size = inputs.shape[0]
#     keep_prob = 1 - p
#     random_tensor = keep_prob
#     random_tensor += torch.rand([batch_size, 1, 1, 1], dtype=inputs.dtype, device=inputs.device)
#     binary_tensor = torch.floor(random_tensor)
#     output = inputs / keep_prob * binary_tensor
#     return output


# def get_same_padding_conv2d(image_size=None):
#      return partial(Conv2dStaticSamePadding, image_size=image_size)

# def get_width_and_height_from_size(x):
#     """ Obtains width and height from a int or tuple """
#     if isinstance(x, int): return x, x
#     if isinstance(x, list) or isinstance(x, tuple): return x
#     else: raise TypeError()

# def calculate_output_image_size(input_image_size, stride):
#     """
#     计算出 Conv2dSamePadding with a stride.
#     """
#     if input_image_size is None: return None
#     image_height, image_width = get_width_and_height_from_size(input_image_size)
#     stride = stride if isinstance(stride, int) else stride[0]
#     image_height = int(math.ceil(image_height / stride))
#     image_width = int(math.ceil(image_width / stride))
#     return [image_height, image_width]



# class Conv2dStaticSamePadding(nn.Conv2d):
#     """ 2D Convolutions like TensorFlow, for a fixed image size"""

#     def __init__(self, in_channels, out_channels, kernel_size, image_size=None, **kwargs):
#         super().__init__(in_channels, out_channels, kernel_size, **kwargs)
#         self.stride = self.stride if len(self.stride) == 2 else [self.stride[0]] * 2

#         # Calculate padding based on image size and save it
#         assert image_size is not None
#         ih, iw = (image_size, image_size) if isinstance(image_size, int) else image_size
#         kh, kw = self.weight.size()[-2:]
#         sh, sw = self.stride
#         oh, ow = math.ceil(ih / sh), math.ceil(iw / sw)
#         pad_h = max((oh - 1) * self.stride[0] + (kh - 1) * self.dilation[0] + 1 - ih, 0)
#         pad_w = max((ow - 1) * self.stride[1] + (kw - 1) * self.dilation[1] + 1 - iw, 0)
#         if pad_h > 0 or pad_w > 0:
#             self.static_padding = nn.ZeroPad2d((pad_w // 2, pad_w - pad_w // 2, pad_h // 2, pad_h - pad_h // 2))
#         else:
#             self.static_padding = Identity()

#     def forward(self, x):
#         x = self.static_padding(x)
#         x = F.conv2d(x, self.weight, self.bias, self.stride, self.padding, self.dilation, self.groups)
#         return x

# class Identity(nn.Module):
#     def __init__(self, ):
#         super(Identity, self).__init__()

#     def forward(self, input):
#         return input


# # MBConvBlock
# class MBConvBlock(nn.Module):
#     '''
#     层 ksize3*3 输入32 输出16  conv1  stride步长1
#     '''
#     def __init__(self, ksize, input_filters, output_filters, expand_ratio=1, stride=1, image_size=224):
#         super().__init__()
#         self._bn_mom = 0.1
#         self._bn_eps = 0.01
#         self._se_ratio = 0.25
#         self._input_filters = input_filters
#         self._output_filters = output_filters
#         self._expand_ratio = expand_ratio
#         self._kernel_size = ksize
#         self._stride = stride

#         inp = self._input_filters
#         oup = self._input_filters * self._expand_ratio
#         if self._expand_ratio != 1:
#             Conv2d = get_same_padding_conv2d(image_size=image_size)
#             self._expand_conv = Conv2d(in_channels=inp, out_channels=oup, kernel_size=1, bias=False)
#             self._bn0 = nn.BatchNorm2d(num_features=oup, momentum=self._bn_mom, eps=self._bn_eps)


#         # Depthwise convolution
#         k = self._kernel_size
#         s = self._stride
#         Conv2d = get_same_padding_conv2d(image_size=image_size)
#         self._depthwise_conv = Conv2d(
#             in_channels=oup, out_channels=oup, groups=oup,
#             kernel_size=k, stride=s, bias=False)
#         self._bn1 = nn.BatchNorm2d(num_features=oup, momentum=self._bn_mom, eps=self._bn_eps)
#         image_size = calculate_output_image_size(image_size, s)

#         # Squeeze and Excitation layer, if desired
#         Conv2d = get_same_padding_conv2d(image_size=(1,1))
#         num_squeezed_channels = max(1, int(self._input_filters * self._se_ratio))
#         self._se_reduce = Conv2d(in_channels=oup, out_channels=num_squeezed_channels, kernel_size=1)
#         self._se_expand = Conv2d(in_channels=num_squeezed_channels, out_channels=oup, kernel_size=1)

#         # Output phase
#         final_oup = self._output_filters
#         Conv2d = get_same_padding_conv2d(image_size=image_size)
#         self._project_conv = Conv2d(in_channels=oup, out_channels=final_oup, kernel_size=1, bias=False)
#         self._bn2 = nn.BatchNorm2d(num_features=final_oup, momentum=self._bn_mom, eps=self._bn_eps)
#         self._swish = MemoryEfficientSwish()

#     def forward(self, inputs, drop_connect_rate=None):
#         """
#         :param inputs: input tensor
#         :param drop_connect_rate: drop connect rate (float, between 0 and 1)
#         :return: output of block
#         """

#         # Expansion and Depthwise Convolution
#         x = inputs
#         if self._expand_ratio != 1:
#             expand = self._expand_conv(inputs)
#             bn0 = self._bn0(expand)
#             x = self._swish(bn0)
#         depthwise = self._depthwise_conv(x)
#         bn1 = self._bn1(depthwise)
#         x = self._swish(bn1)

#         # Squeeze and Excitation
#         x_squeezed = F.adaptive_avg_pool2d(x, 1)
#         x_squeezed = self._se_reduce(x_squeezed)
#         x_squeezed = self._swish(x_squeezed)
#         x_squeezed = self._se_expand(x_squeezed)
#         x = torch.sigmoid(x_squeezed) * x

#         x = self._bn2(self._project_conv(x))

#         # Skip connection and drop connect
#         input_filters, output_filters = self._input_filters, self._output_filters
#         if self._stride == 1 and input_filters == output_filters:
#             if drop_connect_rate:
#                 x = drop_connect(x, p=drop_connect_rate, training=self.training)
#             x = x + inputs  # skip connection
#         return x

# if __name__ == '__main__':
#     input=torch.randn(1,3,112,112)
#     mbconv=MBConvBlock(ksize=3,input_filters=3,output_filters=3,image_size=112)
#     out=mbconv(input)
#     print(out.shape)

In [11]:
# import numpy as np
# import torch
# from torch import nn
# from torch.nn import init



# class ScaledDotProductAttention(nn.Module):
#     '''
#     Scaled dot-product attention
#     '''

#     def __init__(self, d_model, d_k, d_v, h,dropout=.1):
#         '''
#         :param d_model: Output dimensionality of the model
#         :param d_k: Dimensionality of queries and keys
#         :param d_v: Dimensionality of values
#         :param h: Number of heads
#         '''
#         super(ScaledDotProductAttention, self).__init__()
#         self.fc_q = nn.Linear(d_model, h * d_k)
#         self.fc_k = nn.Linear(d_model, h * d_k)
#         self.fc_v = nn.Linear(d_model, h * d_v)
#         self.fc_o = nn.Linear(h * d_v, d_model)
#         self.dropout=nn.Dropout(dropout)

#         self.d_model = d_model
#         self.d_k = d_k
#         self.d_v = d_v
#         self.h = h

#         self.init_weights()


#     def init_weights(self):
#         for m in self.modules():
#             if isinstance(m, nn.Conv2d):
#                 init.kaiming_normal_(m.weight, mode='fan_out')
#                 if m.bias is not None:
#                     init.constant_(m.bias, 0)
#             elif isinstance(m, nn.BatchNorm2d):
#                 init.constant_(m.weight, 1)
#                 init.constant_(m.bias, 0)
#             elif isinstance(m, nn.Linear):
#                 init.normal_(m.weight, std=0.001)
#                 if m.bias is not None:
#                     init.constant_(m.bias, 0)

#     def forward(self, queries, keys, values, attention_mask=None, attention_weights=None):
#         '''
#         Computes
#         :param queries: Queries (b_s, nq, d_model)
#         :param keys: Keys (b_s, nk, d_model)
#         :param values: Values (b_s, nk, d_model)
#         :param attention_mask: Mask over attention values (b_s, h, nq, nk). True indicates masking.
#         :param attention_weights: Multiplicative weights for attention values (b_s, h, nq, nk).
#         :return:
#         '''
#         b_s, nq = queries.shape[:2]
#         nk = keys.shape[1]

#         q = self.fc_q(queries).view(b_s, nq, self.h, self.d_k).permute(0, 2, 1, 3)  # (b_s, h, nq, d_k)
#         k = self.fc_k(keys).view(b_s, nk, self.h, self.d_k).permute(0, 2, 3, 1)  # (b_s, h, d_k, nk)
#         v = self.fc_v(values).view(b_s, nk, self.h, self.d_v).permute(0, 2, 1, 3)  # (b_s, h, nk, d_v)

#         att = torch.matmul(q, k) / np.sqrt(self.d_k)  # (b_s, h, nq, nk)
#         if attention_weights is not None:
#             att = att * attention_weights
#         if attention_mask is not None:
#             att = att.masked_fill(attention_mask, -np.inf)
#         att = torch.softmax(att, -1)
#         att=self.dropout(att)

#         out = torch.matmul(att, v).permute(0, 2, 1, 3).contiguous().view(b_s, nq, self.h * self.d_v)  # (b_s, nq, h*d_v)
#         out = self.fc_o(out)  # (b_s, nq, d_model)
#         return out


# if __name__ == '__main__':
#     input=torch.randn(50,49,512)
#     sa = ScaledDotProductAttention(d_model=512, d_k=512, d_v=512, h=8)
#     output=sa(input,input,input)
#     print(output.shape)

In [12]:
# from torch import nn, sqrt
# import torch
# import sys
# from math import sqrt
# sys.path.append('.')
# # from model.conv.MBConv import MBConvBlock
# # from model.attention.SelfAttention import ScaledDotProductAttention

# class CoAtNet(nn.Module):
#     def __init__(self,in_ch,image_size,out_chs=[8,16,25,40,64]): #[64,96,192,384,768]
#         super().__init__()
#         self.out_chs=out_chs
#         self.maxpool2d=nn.MaxPool2d(kernel_size=2,stride=2)
#         self.maxpool1d = nn.MaxPool1d(kernel_size=2, stride=2)

#         self.s0=nn.Sequential(
#             nn.Conv2d(in_ch,in_ch,kernel_size=3,padding=1),
#             nn.ReLU(),
#             nn.Conv2d(in_ch,in_ch,kernel_size=3,padding=1)
#         )
#         self.mlp0=nn.Sequential(
#             nn.Conv2d(in_ch,out_chs[0],kernel_size=1),
#             nn.ReLU(),
#             nn.Conv2d(out_chs[0],out_chs[0],kernel_size=1)
#         )
        
#         self.s1=MBConvBlock(ksize=3,input_filters=out_chs[0],output_filters=out_chs[0],image_size=image_size//2)
#         self.mlp1=nn.Sequential(
#             nn.Conv2d(out_chs[0],out_chs[1],kernel_size=1),
#             nn.ReLU(),
#             nn.Conv2d(out_chs[1],out_chs[1],kernel_size=1)
#         )

#         # self.s2=MBConvBlock(ksize=3,input_filters=out_chs[1],output_filters=out_chs[1],image_size=image_size//4)
#         # self.mlp2=nn.Sequential(
#         #     nn.Conv2d(out_chs[1],out_chs[2],kernel_size=1),
#         #     nn.ReLU(),
#         #     nn.Conv2d(out_chs[2],out_chs[2],kernel_size=1)
#         # )

#         self.s3=ScaledDotProductAttention(out_chs[1],out_chs[1]//8,out_chs[1]//8,8)
#         self.mlp3=nn.Sequential(
#             nn.Linear(out_chs[1],out_chs[2]),
#             nn.ReLU(),
#             nn.Linear(out_chs[2],out_chs[2])
#         )

#         # self.s4=ScaledDotProductAttention(out_chs[3],out_chs[3]//8,out_chs[3]//8,8)
#         # self.mlp4=nn.Sequential(
#         #     nn.Linear(out_chs[2],out_chs[3]),
#         #     nn.ReLU(),
#         #     nn.Linear(out_chs[3],out_chs[3])
#         # )
        
#         self.Final_layer=nn.Sequential(
#             # nn.Linear(24576, 24576//8),
#             # nn.ReLU(),
#             # nn.Linear(24576//8 , 3072//8),
#             # nn.ReLU(),
#             # nn.Linear(3072//8 , 384//8),
#             # nn.ReLU(),
#             # nn.Linear(384//8 , 2),
#             nn.Linear(12800, 12800//8),
#             nn.ReLU(),
#             nn.Linear(12800//8, 1600//8),
#             nn.ReLU(),
#             nn.Linear(1600//8, 200//8),
#             nn.ReLU(),
#             nn.Linear(200//8, 2),
#             nn.ReLU(),
#             nn.Softmax(dim=1)
#         )

#     def forward(self, x) :
#         B,C,H,W=x.shape
#         #stage0
#         y=self.mlp0(self.s0(x))
#         y=self.maxpool2d(y)
#         #stage1
#         y=self.mlp1(self.s1(y))
#         y=self.maxpool2d(y)
#         #stage2
#         # y=self.mlp2(self.s2(y))
#         # y=self.maxpool2d(y)
#         #stage3
#         # print('###1',y.shape) #[8,64,32,32]
#         y=y.reshape(B,self.out_chs[1],-1).permute(0,2,1) #B,N,C
#         y=self.mlp3(self.s3(y,y,y))
#         y=self.maxpool1d(y.permute(0,2,1)).permute(0,2,1)
#         # print('###2',y.shape)
#         #stage4
#         # y=self.mlp4(self.s4(y,y,y))
#         # y=self.maxpool1d(y.permute(0,2,1))
#         # N=y.shape[-1]
#         # print('###3',y.shape)
#         # y=y.reshape(B,self.out_chs[2],int(sqrt(N)),int(sqrt(N)))
        
#         # print(y.shape)
#         y=torch.flatten(y,1)
        

#         output=self.Final_layer(y)
#         return output

# # if __name__ == '__main__':
# #     x=torch.randn(5,3,128,128)
# #     coatnet=CoAtNet(3,128)
# #     print(coatnet)
# #     y=coatnet(x)
# #     print(y)
# #     print(y.shape)
    

In [13]:
# import timm

# def COVID19Model(pretrained=True):
#     model = timm.create_model('swin_base_patch4_window12_384', pretrained=pretrained)
#     n_features = model.head.in_features  
#     model.classifier = nn.Sequential(nn.Linear(n_features, 2) , nn.Softmax(dim=1))
#     model = model.to(DEVICE) 
#     return model

In [23]:
import torch.nn.functional as F

class custom_CNN(nn.Module):
    def __init__(self, num_classes):
        super(custom_CNN, self).__init__()
        self.layer1=nn.Sequential(
                        nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3),
                        nn.ReLU(),
                        nn.MaxPool2d(kernel_size=2,stride=2),
                        nn.BatchNorm2d(32)
                    )
        
        self.layer2=nn.Sequential(
                        nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3),
                        nn.ReLU(),
                        nn.MaxPool2d(kernel_size=2,stride=2),
                        nn.BatchNorm2d(32)
                    )
        
        self.layer3=nn.Sequential(
                        nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3),
                        nn.ReLU(),
                        nn.MaxPool2d(kernel_size=2,stride=2),
                        nn.BatchNorm2d(64)
                    )
        
        self.layer4=nn.Sequential(
                        nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3),
                        nn.ReLU(),
                        nn.MaxPool2d(kernel_size=2,stride=2),
                        nn.BatchNorm2d(64)
                    )
        
        self.fc1 = nn.Sequential(
                        nn.Linear(in_features=64*14*14, out_features=64*14),
                        nn.Dropout(p=0.3)
                    )
        self.fc2 = nn.Sequential(
                        nn.Linear(in_features=64*14, out_features=128),
                        nn.Dropout(p=0.3)
                    )
        self.fc3 = nn.Sequential(
                        nn.Linear(in_features=128, out_features=num_classes),
                        nn.Dropout(p=0.3)
                    )
        self.softmax = nn.Softmax(dim=1)
        
    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        
        # print(x.shape)
        x = torch.flatten(x,1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        
        output = self.softmax(x)
        # print(output)
        return output

if __name__ == '__main__':
    x=torch.randn(5,3,128,128)
    model=custom_CNN(2)

    y=model(x)
    print(y)
    print(y.shape)

tensor([[0.4564, 0.5436],
        [0.4397, 0.5603],
        [0.4968, 0.5032],
        [0.4997, 0.5003],
        [0.5366, 0.4634]], grad_fn=<SoftmaxBackward>)
torch.Size([5, 2])


## Utils
### EarlyStopper

In [24]:
class LossEarlyStopper():
    """Early stopper
    
    Attributes:
        patience (int): loss가 줄어들지 않아도 학습할 epoch 수
        patience_counter (int): loss 가 줄어들지 않을 때 마다 1씩 증가, 감소 시 0으로 리셋
        min_loss (float): 최소 loss
        stop (bool): True 일 때 학습 중단

    """

    def __init__(self, patience: int)-> None:
        self.patience = patience

        self.patience_counter = 0
        self.min_loss = np.Inf
        self.stop = False
        self.save_model = False

    def check_early_stopping(self, loss: float)-> None:
        """Early stopping 여부 판단"""  

        if self.min_loss == np.Inf:
            self.min_loss = loss
            return None

        elif loss > self.min_loss:
            self.patience_counter += 1
            msg = f"Early stopping counter {self.patience_counter}/{self.patience}"

            if self.patience_counter == self.patience:
                self.stop = True
                
        elif loss <= self.min_loss:
            self.patience_counter = 0
            self.save_model = True
            msg = f"Validation loss decreased {self.min_loss} -> {loss}"
            self.min_loss = loss
        
        # print(msg)

### Trainer

In [25]:
class Trainer():
    """ epoch에 대한 학습 및 검증 절차 정의"""
    
    def __init__(self, loss_fn, model, device, metric_fn, optimizer=None, scheduler=None):
        """ 초기화
        """
        self.loss_fn = loss_fn
        self.model = model
        self.device = device
        self.optimizer = optimizer
        self.scheduler = scheduler
        self.metric_fn = metric_fn

    def train_epoch(self, dataloader, epoch_index):
        """ 한 epoch에서 수행되는 학습 절차"""
        
        self.model.train()
        train_total_loss = 0
        target_lst = []
        pred_lst = []
        prob_lst = []

        for batch_index, (img, label) in enumerate(dataloader):
            img = img.to(self.device)
            label = label.to(self.device).float()
            
            pred = self.model(img)
            
            loss = self.loss_fn(pred[:,1], label)
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
            self.scheduler.step()
            
            train_total_loss += loss.item()
            prob_lst.extend(pred[:, 1].cpu().tolist())
            target_lst.extend(label.cpu().tolist())
            pred_lst.extend(pred.argmax(dim=1).cpu().tolist())
            wandb.log({"train_loss": loss})
        self.train_mean_loss = train_total_loss / batch_index
        self.train_score, f1 = self.metric_fn(y_pred=pred_lst, y_answer=target_lst)
        msg = f'Epoch {epoch_index}, Train loss: {self.train_mean_loss}, Acc: {self.train_score}, F1-Macro: {f1}'
        print(msg)

    def validate_epoch(self, dataloader, epoch_index):
        """ 한 epoch에서 수행되는 검증 절차
        """
        self.model.eval()
        val_total_loss = 0
        target_lst = []
        pred_lst = []
        prob_lst = []

        for batch_index, (img, label) in enumerate(dataloader):
            img = img.to(self.device)
            label = label.to(self.device).float()
            pred = self.model(img)
            
            loss = self.loss_fn(pred[:,1], label)
            val_total_loss += loss.item()
            prob_lst.extend(pred[:, 1].cpu().tolist())
            target_lst.extend(label.cpu().tolist())
            pred_lst.extend(pred.argmax(dim=1).cpu().tolist())
            wandb.log({"valid_loss": loss})
        self.val_mean_loss = val_total_loss / batch_index
        self.validation_score, f1 = self.metric_fn(y_pred=pred_lst, y_answer=target_lst)
        msg = f'Epoch {epoch_index}, Val loss: {self.val_mean_loss}, Acc: {self.validation_score}, F1-Macro: {f1}'
        print(msg)

### Metrics

In [26]:
from sklearn.metrics import accuracy_score, f1_score

def get_metric_fn(y_pred, y_answer):
    """ 성능을 반환하는 함수"""
    
    assert len(y_pred) == len(y_answer), 'The size of prediction and answer are not same.'
    accuracy = accuracy_score(y_answer, y_pred)
    f1 = f1_score(y_answer, y_pred, average='macro')
    return accuracy, f1

## Train
### 학습을 위한 객체 선언

#### Load Dataset & Dataloader

In [27]:
# Load dataset & dataloader
train_dataset = CustomDataset(data_dir=DATA_DIR, mode='train', input_shape=INPUT_SHAPE)
validation_dataset = CustomDataset(data_dir=DATA_DIR, mode='val', input_shape=INPUT_SHAPE)
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
validation_dataloader = DataLoader(validation_dataset, batch_size=BATCH_SIZE, shuffle=True)
print('Train set samples:',len(train_dataset),  'Val set samples:', len(validation_dataset))
# print(train_dataset[1][0].shape)

Loading train dataset..
Loading val dataset..
Train set samples: 581 Val set samples: 65


#### Load model and other utils

In [28]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [29]:
# Load Model
model = custom_CNN(NUM_CLS).to(DEVICE)
# model = CoAtNet(3, 128).to(DEVICE)
# model = COVID19Model(pretrained=False)

# # Save Initial Model
# torch.save(model.state_dict(), 'initial.pt')

# Set optimizer, scheduler, loss function, metric function
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
scheduler =  optim.lr_scheduler.OneCycleLR(optimizer=optimizer, pct_start=0.1, div_factor=1e5, max_lr=0.0001, epochs=EPOCHS, steps_per_epoch=len(train_dataloader))
loss_fn = nn.BCELoss()
metric_fn = get_metric_fn


# Set trainer
trainer = Trainer(loss_fn, model, DEVICE, metric_fn, optimizer, scheduler)

# Set earlystopper
early_stopper = LossEarlyStopper(patience=EARLY_STOPPING_PATIENCE)

In [30]:
model

custom_CNN(
  (layer1): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (layer2): Sequential(
    (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (layer3): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (layer4): Sequential(
    (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilatio

### epoch 단위 학습 진행

In [31]:
for epoch_index in tqdm(range(EPOCHS)):

    trainer.train_epoch(train_dataloader, epoch_index)
    trainer.validate_epoch(validation_dataloader, epoch_index)

    # early_stopping check
    early_stopper.check_early_stopping(loss=trainer.val_mean_loss)

    if early_stopper.stop:
        print('Early stopped')
        break

    if early_stopper.save_model:
        check_point = {
            'model': model.state_dict(),
            'optimizer': optimizer.state_dict(),
            'scheduler': scheduler.state_dict()
        }
        torch.save(check_point, 'best.pt')

  0% 0/50 [00:00<?, ?it/s]

Epoch 0, Train loss: 0.7290441791216532, Acc: 0.5559380378657487, F1-Macro: 0.4672982885085575


  2% 1/50 [01:00<49:21, 60.45s/it]

Epoch 0, Val loss: 1.04808709025383, Acc: 0.49230769230769234, F1-Macro: 0.32989690721649484
Epoch 1, Train loss: 0.6799141532844968, Acc: 0.6333907056798623, F1-Macro: 0.6056875209100033
Epoch 1, Val loss: 1.028282105922699, Acc: 0.5538461538461539, F1-Macro: 0.48342011510002736


  4% 2/50 [02:16<52:03, 65.08s/it]

...

Epoch 15, Train loss: 0.146047154131035, Acc: 0.9070567986230637, F1-Macro: 0.9046127839665321
Epoch 15, Val loss: 0.5508373202756047, Acc: 0.7384615384615385, F1-Macro: 0.7374673319078165


 32% 16/50 [16:20<33:57, 59.93s/it]

Epoch 16, Train loss: 0.1479226876464155, Acc: 0.9018932874354562, F1-Macro: 0.8992562172028291


 32% 16/50 [17:19<36:49, 64.98s/it]

Epoch 16, Val loss: 0.9211353063583374, Acc: 0.7538461538461538, F1-Macro: 0.7523809523809524
Early stopped


## Inference
### 모델 로드

In [32]:
TRAINED_MODEL_PATH = 'best.pt'

### Load dataset

In [33]:
class TestDataset(Dataset):
    def __init__(self, data_dir, input_shape):
        self.data_dir = data_dir
        self.input_shape = input_shape
        
        # Loading dataset
        self.db = self.data_loader()
        
        # Transform function
        self.transform = transforms.Compose([transforms.Resize(self.input_shape),
                                             transforms.ToTensor(),
                                             transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

    def data_loader(self):
        print('Loading test dataset..')
        if not os.path.isdir(self.data_dir):
            print(f'!!! Cannot find {self.data_dir}... !!!')
            sys.exit()
        
        db = pd.read_csv(os.path.join(self.data_dir, 'sample_submission.csv'))
        return db
    
    def __len__(self):
        return len(self.db)
    
    def __getitem__(self, index):
        data = copy.deepcopy(self.db.loc[index])
        
        # Loading image
        cvimg = cv2.imread(os.path.join(self.data_dir,'test',data['file_name']), cv2.IMREAD_COLOR | cv2.IMREAD_IGNORE_ORIENTATION)
        if not isinstance(cvimg, np.ndarray):
            raise IOError("Fail to read %s" % data['file_name'])

        # Preprocessing images
        trans_image = self.transform(Image.fromarray(cvimg))

        return trans_image, data['file_name']

In [34]:
# Load dataset & dataloader
test_dataset = TestDataset(data_dir=DATA_DIR, input_shape=INPUT_SHAPE)
test_dataloader = DataLoader(dataset=test_dataset, batch_size=BATCH_SIZE, shuffle=False)

Loading test dataset..


### 추론 진행

In [35]:
model.load_state_dict(torch.load(TRAINED_MODEL_PATH)['model'])

# Prediction
file_lst = []
pred_lst = []
prob_lst = []
model.eval()
with torch.no_grad():
    for batch_index, (img, file_num) in tqdm(enumerate(test_dataloader)):
        img = img.to(DEVICE)
        pred = model(img)
        print(pred)
        file_lst.extend(list(file_num))
        pred_lst.extend(pred.argmax(dim=1).tolist())
        prob_lst.extend(pred[:, 1].tolist())

1it [00:02,  2.47s/it]

tensor([[9.7230e-01, 2.7700e-02],
        [8.4935e-01, 1.5065e-01],
        [8.1651e-01, 1.8349e-01],
        [3.4291e-01, 6.5709e-01],
        [9.8791e-01, 1.2090e-02],
        [7.4627e-03, 9.9254e-01],
        [6.6846e-01, 3.3154e-01],
        [7.5879e-02, 9.2412e-01],
        [9.8790e-01, 1.2102e-02],
        [1.8778e-05, 9.9998e-01],
        [9.7802e-01, 2.1982e-02],
        [9.3440e-01, 6.5602e-02],
        [9.8545e-01, 1.4554e-02],
        [9.5419e-01, 4.5813e-02],
        [9.0247e-01, 9.7534e-02],
        [9.8905e-01, 1.0951e-02],
        [4.4628e-01, 5.5372e-01],
        [9.9488e-01, 5.1226e-03],
        [4.7339e-01, 5.2661e-01],
        [9.8920e-01, 1.0798e-02],
        [9.9853e-01, 1.4723e-03],
        [9.9292e-01, 7.0771e-03],
        [9.2529e-01, 7.4712e-02],
        [9.6305e-01, 3.6955e-02],
        [9.6643e-01, 3.3571e-02],
        [9.9677e-01, 3.2302e-03],
        [9.9283e-01, 7.1667e-03],
        [3.0243e-01, 6.9757e-01],
        [4.9650e-03, 9.9504e-01],
        [1.071

2it [00:05,  2.51s/it]

tensor([[0.1690, 0.8310],
        [0.9627, 0.0373],
        [0.9866, 0.0134],
        [0.2451, 0.7549],
        [0.6401, 0.3599],
        [0.0494, 0.9506],
        [0.0623, 0.9377],
        [0.0279, 0.9721],
        [0.0047, 0.9953],
        [0.5879, 0.4121],
        [0.9903, 0.0097],
        [0.0054, 0.9946],
        [0.9679, 0.0321],
        [0.0144, 0.9856],
        [0.0157, 0.9843],
        [0.9964, 0.0036],
        [0.9759, 0.0241],
        [0.8684, 0.1316],
        [0.0302, 0.9698],
        [0.0030, 0.9970],
        [0.2906, 0.7094],
        [0.9656, 0.0344],
        [0.0278, 0.9722],
        [0.7928, 0.2072],
        [0.5523, 0.4477],
        [0.0060, 0.9940],
        [0.9895, 0.0105],
        [0.7135, 0.2865],
        [0.0092, 0.9908],
        [0.9496, 0.0504],
        [0.9962, 0.0038],
        [0.9905, 0.0095]])


3it [00:07,  2.45s/it]

tensor([[9.7271e-03, 9.9027e-01],
        [4.7026e-04, 9.9953e-01],
        [9.1508e-01, 8.4919e-02],
        [9.6868e-01, 3.1316e-02],
        [9.2225e-03, 9.9078e-01],
        [9.8523e-01, 1.4766e-02],
        [9.9175e-01, 8.2498e-03],
        [9.9114e-01, 8.8600e-03],
        [9.7607e-01, 2.3931e-02],
        [9.9221e-01, 7.7887e-03],
        [6.3900e-02, 9.3610e-01],
        [9.9521e-01, 4.7896e-03],
        [9.9834e-01, 1.6568e-03],
        [9.9659e-01, 3.4119e-03],
        [4.6501e-01, 5.3499e-01],
        [3.6431e-03, 9.9636e-01],
        [9.3479e-01, 6.5208e-02],
        [9.2841e-01, 7.1593e-02],
        [9.6488e-01, 3.5123e-02],
        [1.7765e-02, 9.8223e-01],
        [3.5839e-02, 9.6416e-01],
        [8.6134e-01, 1.3866e-01],
        [6.9404e-01, 3.0596e-01],
        [9.1473e-01, 8.5270e-02],
        [9.8796e-01, 1.2040e-02],
        [1.1598e-01, 8.8402e-01],
        [9.9233e-01, 7.6669e-03],
        [9.8980e-01, 1.0197e-02],
        [3.0205e-01, 6.9795e-01],
        [1.220

4it [00:07,  1.94s/it]

tensor([[0.0061, 0.9939],
        [0.2433, 0.7567],
        [0.0153, 0.9847],
        [0.0127, 0.9873]])


### 결과 저장

In [36]:
df = pd.DataFrame({'file_name':file_lst, 'COVID':pred_lst})
# df.sort_values(by=['file_name'], inplace=True)
df.to_csv('prediction.csv', index=False)

In [37]:
file_lst = []
pred_lst = []
prob_lst = []
model.eval()
with torch.no_grad():
    for batch_index, (img, file_num) in tqdm(enumerate(test_dataloader)):
        img = img.to(DEVICE)
        pred = model(img)
        print(pred)
        file_lst.extend(list(file_num))
        pred_lst.extend(pred.tolist())
df = pd.DataFrame(pred_lst)
# df.sort_values(by=['file_name'], inplace=True)
df.to_csv('prediction111.csv', index=False)

1it [00:02,  2.39s/it]

tensor([[9.7230e-01, 2.7700e-02],
        [8.4935e-01, 1.5065e-01],
        [8.1651e-01, 1.8349e-01],
        [3.4291e-01, 6.5709e-01],
        [9.8791e-01, 1.2090e-02],
        [7.4627e-03, 9.9254e-01],
        [6.6846e-01, 3.3154e-01],
        [7.5879e-02, 9.2412e-01],
        [9.8790e-01, 1.2102e-02],
        [1.8778e-05, 9.9998e-01],
        [9.7802e-01, 2.1982e-02],
        [9.3440e-01, 6.5602e-02],
        [9.8545e-01, 1.4554e-02],
        [9.5419e-01, 4.5813e-02],
        [9.0247e-01, 9.7534e-02],
        [9.8905e-01, 1.0951e-02],
        [4.4628e-01, 5.5372e-01],
        [9.9488e-01, 5.1226e-03],
        [4.7339e-01, 5.2661e-01],
        [9.8920e-01, 1.0798e-02],
        [9.9853e-01, 1.4723e-03],
        [9.9292e-01, 7.0771e-03],
        [9.2529e-01, 7.4712e-02],
        [9.6305e-01, 3.6955e-02],
        [9.6643e-01, 3.3571e-02],
        [9.9677e-01, 3.2302e-03],
        [9.9283e-01, 7.1667e-03],
        [3.0243e-01, 6.9757e-01],
        [4.9650e-03, 9.9504e-01],
        [1.071

2it [00:04,  2.45s/it]

tensor([[0.1690, 0.8310],
        [0.9627, 0.0373],
        [0.9866, 0.0134],
        [0.2451, 0.7549],
        [0.6401, 0.3599],
        [0.0494, 0.9506],
        [0.0623, 0.9377],
        [0.0279, 0.9721],
        [0.0047, 0.9953],
        [0.5879, 0.4121],
        [0.9903, 0.0097],
        [0.0054, 0.9946],
        [0.9679, 0.0321],
        [0.0144, 0.9856],
        [0.0157, 0.9843],
        [0.9964, 0.0036],
        [0.9759, 0.0241],
        [0.8684, 0.1316],
        [0.0302, 0.9698],
        [0.0030, 0.9970],
        [0.2906, 0.7094],
        [0.9656, 0.0344],
        [0.0278, 0.9722],
        [0.7928, 0.2072],
        [0.5523, 0.4477],
        [0.0060, 0.9940],
        [0.9895, 0.0105],
        [0.7135, 0.2865],
        [0.0092, 0.9908],
        [0.9496, 0.0504],
        [0.9962, 0.0038],
        [0.9905, 0.0095]])


3it [00:07,  2.47s/it]

tensor([[9.7271e-03, 9.9027e-01],
        [4.7026e-04, 9.9953e-01],
        [9.1508e-01, 8.4919e-02],
        [9.6868e-01, 3.1316e-02],
        [9.2225e-03, 9.9078e-01],
        [9.8523e-01, 1.4766e-02],
        [9.9175e-01, 8.2498e-03],
        [9.9114e-01, 8.8600e-03],
        [9.7607e-01, 2.3931e-02],
        [9.9221e-01, 7.7887e-03],
        [6.3900e-02, 9.3610e-01],
        [9.9521e-01, 4.7896e-03],
        [9.9834e-01, 1.6568e-03],
        [9.9659e-01, 3.4119e-03],
        [4.6501e-01, 5.3499e-01],
        [3.6431e-03, 9.9636e-01],
        [9.3479e-01, 6.5208e-02],
        [9.2841e-01, 7.1593e-02],
        [9.6488e-01, 3.5123e-02],
        [1.7765e-02, 9.8223e-01],
        [3.5839e-02, 9.6416e-01],
        [8.6134e-01, 1.3866e-01],
        [6.9404e-01, 3.0596e-01],
        [9.1473e-01, 8.5270e-02],
        [9.8796e-01, 1.2040e-02],
        [1.1598e-01, 8.8402e-01],
        [9.9233e-01, 7.6669e-03],
        [9.8980e-01, 1.0197e-02],
        [3.0205e-01, 6.9795e-01],
        [1.220

4it [00:07,  1.97s/it]

tensor([[0.0061, 0.9939],
        [0.2433, 0.7567],
        [0.0153, 0.9847],
        [0.0127, 0.9873]])
